In [18]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=10,
    horizontal_flip=True,
    zoom_range=0.1,
    shear_range=0.1
)



train_generator = train_datagen.flow_from_directory(
    'F:/Project/Final Dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'F:/Project/Final Dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 18677 images belonging to 2 classes.
Found 4668 images belonging to 2 classes.


In [19]:
base_model = tf.keras.applications.InceptionV3(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')

In [20]:
# Freeze the base model layers
base_model.trainable = False

In [21]:
# Add custom layers on top of the base model for classification
model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [22]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 flatten_2 (Flatten)         (None, 51200)             0         
                                                                 
 dense_4 (Dense)             (None, 128)               6553728   
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 2)                 258       
                                                                 
Total params: 28,356,770
Trainable params: 6,553,986
Non-trainable params: 21,802,784
_________________________________________________________________


In [24]:
# Set up early stopping
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [25]:
history = model.fit(train_generator,
                    epochs=10,
                    validation_data=validation_generator,
                    callbacks=[early_stop])


Epoch 1/10
584/584 [==============================] - 716s 1s/step - loss: 0.3285 - accuracy: 0.9381 - val_loss: 0.0628 - val_accuracy: 0.9809
Epoch 2/10
584/584 [==============================] - 745s 1s/step - loss: 0.1200 - accuracy: 0.9565 - val_loss: 0.0401 - val_accuracy: 0.9893
Epoch 3/10
584/584 [==============================] - 738s 1s/step - loss: 0.0939 - accuracy: 0.9656 - val_loss: 0.0529 - val_accuracy: 0.9846
Epoch 4/10
584/584 [==============================] - 735s 1s/step - loss: 0.0970 - accuracy: 0.9613 - val_loss: 0.0368 - val_accuracy: 0.9882
Epoch 5/10
584/584 [==============================] - 747s 1s/step - loss: 0.0868 - accuracy: 0.9613 - val_loss: 0.0494 - val_accuracy: 0.9835
Epoch 6/10
584/584 [==============================] - 737s 1s/step - loss: 0.0744 - accuracy: 0.9716 - val_loss: 0.0449 - val_accuracy: 0.9826
Epoch 7/10
584/584 [==============================] - 737s 1s/step - loss: 0.0715 - accuracy: 0.9748 - val_loss: 0.0341 - val_accuracy: 0.9880

In [27]:
history = model.fit(train_generator,
                    epochs=15,
                    validation_data=validation_generator,
                    callbacks=[early_stop],
                    initial_epoch=10)


Epoch 11/15
584/584 [==============================] - 660s 1s/step - loss: 0.0688 - accuracy: 0.9723 - val_loss: 0.0291 - val_accuracy: 0.9906
Epoch 12/15
584/584 [==============================] - 835s 1s/step - loss: 0.0690 - accuracy: 0.9669 - val_loss: 0.0388 - val_accuracy: 0.9869
Epoch 13/15
584/584 [==============================] - 1028s 2s/step - loss: 0.0671 - accuracy: 0.9729 - val_loss: 0.0448 - val_accuracy: 0.9837
Epoch 14/15
584/584 [==============================] - 660s 1s/step - loss: 0.0556 - accuracy: 0.9785 - val_loss: 0.0316 - val_accuracy: 0.9897


In [28]:
model.save('project17_InceptionV3model.h5')